In [3]:
library(readxl)

In [4]:
library(sandwich)
library(msm)

In [5]:
df = read_excel('CountsData.xlsx',sheet = 'poisson')

In [6]:
head(df,10)

num_awards,prog,math
<dbl>,<dbl>,<dbl>
0,3,41
0,1,41
0,3,44
0,3,42
0,3,40
0,1,42
0,3,46
0,3,40
0,3,33


- program is a categorical data( 1,2,3 is type of program)
- math is a continuous data(41,44,.. etc are scores)

In [7]:
unique(df$num_awards)

[1] 0 1 3 2 5 4 6

In [8]:
var(df$prog)

[1] 0.4767588

In [9]:
var(df$num_awards)

[1] 1.108643

In [10]:
str(df)

tibble [200 x 3] (S3: tbl_df/tbl/data.frame)
 $ num_awards: num [1:200] 0 0 0 0 0 0 0 0 0 0 ...
 $ prog      : num [1:200] 3 1 3 3 3 1 3 3 3 3 ...
 $ math      : num [1:200] 41 41 44 42 40 42 46 40 33 46 ...


In [11]:
df$prog = as.factor(df$prog)

In [12]:
str(df)

tibble [200 x 3] (S3: tbl_df/tbl/data.frame)
 $ num_awards: num [1:200] 0 0 0 0 0 0 0 0 0 0 ...
 $ prog      : Factor w/ 3 levels "1","2","3": 3 1 3 3 3 1 3 3 3 3 ...
 $ math      : num [1:200] 41 41 44 42 40 42 46 40 33 46 ...


In [13]:
length(which(df$num_awards == 0))

[1] 124

In [14]:
length(which(df$num_awards == 0|df$num_awards == 1))

[1] 173

In [15]:
unique(df$num_awards)

[1] 0 1 3 2 5 4 6

In [16]:
df[which(df$num_awards == 5),]

num_awards,prog,math
<dbl>,<fct>,<dbl>
5,2,61
5,2,71


In [17]:
mod = glm(num_awards ~ prog + math, data = df, family = 'poisson')

In [18]:
summary(mod)


Call:
glm(formula = num_awards ~ prog + math, family = "poisson", data = df)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2043  -0.8436  -0.5106   0.2558   2.6796  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -5.24712    0.65845  -7.969 1.60e-15 ***
prog2        1.08386    0.35825   3.025  0.00248 ** 
prog3        0.36981    0.44107   0.838  0.40179    
math         0.07015    0.01060   6.619 3.63e-11 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 287.67  on 199  degrees of freedom
Residual deviance: 189.45  on 196  degrees of freedom
AIC: 373.5

Number of Fisher Scoring iterations: 6


In [21]:
data = data.frame(prog=c('1','2','3'),math = 60)

In [22]:
predict(mod,data,type = 'response')

1         2         3 
0.3541692 1.0469473 0.5126452

In [23]:
data = data.frame(prog=c('2'),math = c(60,85))

In [24]:
predict(mod,data,type = 'response')

1        2 
1.046947 6.047763

In [25]:
library(MASS)

When counts in the response are over dispersed(variation is high. Use negative binomial regression

In [26]:
df = read_excel('CountsData.xlsx',sheet = 'neg_bin')

In [27]:
head(df)

math,prog,daysabs
<dbl>,<chr>,<dbl>
63,Academic,4
27,Academic,4
20,Academic,2
16,Academic,3
2,Academic,3
71,Academic,13


In [29]:
df$prog = as.factor(df$prog)

In [30]:
str(df)

tibble [314 x 3] (S3: tbl_df/tbl/data.frame)
 $ math   : num [1:314] 63 27 20 16 2 71 63 3 51 49 ...
 $ prog   : Factor w/ 3 levels "Academic","General",..: 1 1 1 1 1 1 1 1 1 3 ...
 $ daysabs: num [1:314] 4 4 2 3 3 13 11 7 10 9 ...


In [32]:
sd(df$daysabs)

[1] 7.036958

In [33]:
mod = glm.nb(daysabs ~ .,data = df)

In [34]:
summary(mod)


Call:
glm.nb(formula = daysabs ~ ., data = df, init.theta = 1.032713156, 
    link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1547  -1.0192  -0.3694   0.2285   2.5273  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)     2.174505   0.133411  16.299  < 2e-16 ***
math           -0.005993   0.002505  -2.392   0.0167 *  
progGeneral     0.440760   0.182610   2.414   0.0158 *  
progVocational -0.837891   0.144087  -5.815 6.06e-09 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(1.0327) family taken to be 1)

    Null deviance: 427.54  on 313  degrees of freedom
Residual deviance: 358.52  on 310  degrees of freedom
AIC: 1741.3

Number of Fisher Scoring iterations: 1


              Theta:  1.033 
          Std. Err.:  0.106 

 2 x log-likelihood:  -1731.258 

In [35]:
data = data.frame(prog = c('Academic','General','Vocational'),math = 60)

In [36]:
predict(mod,data,type = 'response')

1        2        3 
6.140623 9.541838 2.656569

In [39]:
df$prog = relevel(df$prog,ref = 'Academic')

In [40]:
mod = glm.nb(daysabs ~ .,data = df)

In [41]:
summary(mod)


Call:
glm.nb(formula = daysabs ~ ., data = df, init.theta = 1.032713156, 
    link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.1547  -1.0192  -0.3694   0.2285   2.5273  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)     2.174505   0.133411  16.299  < 2e-16 ***
math           -0.005993   0.002505  -2.392   0.0167 *  
progGeneral     0.440760   0.182610   2.414   0.0158 *  
progVocational -0.837891   0.144087  -5.815 6.06e-09 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(1.0327) family taken to be 1)

    Null deviance: 427.54  on 313  degrees of freedom
Residual deviance: 358.52  on 310  degrees of freedom
AIC: 1741.3

Number of Fisher Scoring iterations: 1


              Theta:  1.033 
          Std. Err.:  0.106 

 2 x log-likelihood:  -1731.258 

In [42]:
data = data.frame(prog = 'General',math = c(40,50,60,80))

In [43]:
predict(mod,data,type = 'response')

1         2         3         4 
10.756884 10.131162  9.541838  8.464038